---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the Jupyter Notebook FAQ course resource._

---

# Assignment 4 - Predicting and understanding viewer engagement with educational videos 

With the accelerating popularity of online educational experiences, the role of online lectures and other educational video continues to increase in scope and importance. Open access educational repositories such as <a href="http://videolectures.net/">videolectures.net</a>, as well as Massive Open Online Courses (MOOCs) on platforms like Coursera, have made access to many thousands of lectures and tutorials an accessible option for millions of people around the world. Yet this impressive volume of content has also led to a challenge in how to find, filter, and match these videos with learners. This assignment gives you an example of how machine learning can be used to address part of that challenge.

## About the prediction problem

One critical property of a video is engagement: how interesting or "engaging" it is for viewers, so that they decide to keep watching. Engagement is critical for learning, whether the instruction is coming from a video or any other source. There are many ways to define engagement with video, but one common approach is to estimate it by measuring how much of the video a user watches. If the video is not interesting and does not engage a viewer, they will typically abandon it quickly, e.g. only watch 5 or 10% of the total. 

A first step towards providing the best-matching educational content is to understand which features of educational material make it engaging for learners in general. This is where predictive modeling can be applied, via supervised machine learning. For this assignment, your task is to predict how engaging an educational video is likely to be for viewers, based on a set of features extracted from the video's transcript, audio track, hosting site, and other sources.

We chose this prediction problem for several reasons:

* It combines a variety of features derived from a rich set of resources connected to the original data;
* The manageable dataset size means the dataset and supervised models for it can be easily explored on a wide variety of computing platforms;
* Predicting popularity or engagement for a media item, especially combined with understanding which features contribute to its success with viewers, is a fun problem but also a practical representative application of machine learning in a number of business and educational sectors.


## About the dataset

We extracted training and test datasets of educational video features from the VLE Dataset put together by researcher Sahan Bulathwela at University College London. 

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single educational video, and includes information about diverse properties of the video content as described further below. The target variable is `engagement` which was defined as True if the median percentage of the video watched across all viewers was at least 30%, and False otherwise.

Note: Any extra variables that may be included in the training set are simply for your interest if you want an additional source of data for visualization, or to enable unsupervised and semi-supervised approaches. However, they are not included in the test set and thus cannot be used for prediction. **Only the data already included in your Coursera directory can be used for training the model for this assignment.**

For this final assignment, you will bring together what you've learned across all four weeks of this course, by exploring different prediction models for this new dataset. In addition, we encourage you to apply what you've learned about model selection to do hyperparameter tuning using training/validation splits of the training data, to optimize the model and further increase its performance. In addition to a basic evaluation of model accuracy, we've also provided a utility function to visualize which features are most and least contributing to the overall model performance.

**File descriptions** 
    assets/train.csv - the training set (Use only this data for training your model!)
    assets/test.csv - the test set
<br>

**Data fields**

train.csv & test.csv:

    title_word_count - the number of words in the title of the video.
    
    document_entropy - a score indicating how varied the topics are covered in the video, based on the transcript. Videos with smaller entropy scores will tend to be more cohesive and more focused on a single topic.
    
    freshness - The number of days elapsed between 01/01/1970 and the lecture published date. Videos that are more recent will have higher freshness values.
    
    easiness - A text difficulty measure applied to the transcript. A lower score indicates more complex language used by the presenter.
    
    fraction_stopword_presence - A stopword is a very common word like 'the' or 'and'. This feature computes the fraction of all words that are stopwords in the video lecture transcript.
    
    speaker_speed - The average speaking rate in words per minute of the presenter in the video.
    
    silent_period_rate - The fraction of time in the lecture video that is silence (no speaking).
    
train.csv only:
    
    engagement - Target label for training. True if learners watched a substantial portion of the video (see description), or False otherwise.
    

## Evaluation

Your predictions will be given as the probability that the corresponding video will be engaging to learners.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model with an AUC (area under ROC curve) of at least 0.8 passes this assignment, and over 0.85 will receive full points.
___

For this assignment, create a function that trains a model to predict significant learner engagement with a video using `asset/train.csv`. Using this model, return a Pandas Series object of length 2309 with the data being the probability that each corresponding video from `readonly/test.csv` will be engaging (according to a model learned from the 'engagement' label in the training set), and the video index being in the `id` field.

Example:

    id
       9240    0.401958
       9241    0.105928
       9242    0.018572
                 ...
       9243    0.208567
       9244    0.818759
       9245    0.018528
             ...
       Name: engagement, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out and check your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. 

* Try to avoid global variables. If you have other functions besides engagement_model, you should move those functions inside the scope of engagement_model.

* Be sure to first check the pinned threads in Week 4's discussion forum if you run into a problem you can't figure out.

### Extensions

* If this prediction task motivates you to explore further, you can find more details here on the original VLE dataset and others related to video engagement: https://github.com/sahanbull/VLE-Dataset



In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)   # Do not change this value: required to be compatible with solutions generated by the autograder.

In [12]:
# exprimentation
train = pd.read_csv("assets/train.csv")
test = pd.read_csv("assets/test.csv")
x_train = train.drop(["engagement"], axis=1)
y_train = np.array(train["engagement"].values, dtype=np.int16)

array([1, 0, 0, ..., 0, 0, 1], dtype=int16)

In [6]:
test_set.head()

,id,title_word_count,document_entropy,freshness,easiness,fraction_stopword_presence,normalization_rate,speaker_speed,silent_period_rate
0,9240,6,8.548351,14140,89.827395,0.640810,0.017945,2.262723,0.000000
1,9241,8,7.730110,14600,82.446667,0.606738,0.027708,2.690351,0.000000
2,9242,3,8.200887,16980,88.821542,0.621089,0.009857,3.116071,0.000000
3,9243,5,6.377299,16260,86.874660,0.600000,0.004348,2.837500,0.017994
4,9244,18,7.756530,14030,88.872277,0.616105,0.033240,1.354839,0.000000


In [14]:
test_set.describe()

,id,title_word_count,document_entropy,freshness,easiness,fraction_stopword_presence,normalization_rate,speaker_speed,silent_period_rate
count,2309.000000,2309.000000,2309.000000,2309.000000,2309.000000,2309.000000,2309.000000,2309.000000,2309.000000
mean,10394.000000,7.721958,7.786447,14861.082720,84.629109,0.613106,0.021450,2.824541,0.145908
std,666.695208,3.736182,0.690601,1186.249075,8.437686,0.049835,0.009962,18.372017,0.174642
min,9240.000000,1.000000,0.574823,10850.000000,35.743571,0.130435,0.000000,0.002373,0.000000
25%,9817.000000,5.000000,7.575573,14070.000000,80.114882,0.588602,0.014849,1.964286,0.000000
50%,10394.000000,7.000000,7.881753,14750.000000,84.239102,0.613808,0.019878,2.260714,0.094263
75%,10971.000000,10.000000,8.169210,15620.000000,88.310296,0.635729,0.026099,2.544403,0.251005
max,11548.000000,28.000000,9.144478,17430.000000,121.205073,0.982114,0.101655,881.000000,1.542962


In [47]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, auc, roc_curve

def feature_selection(X, y):
    """ performes feature selection using decision tree classifier"""
    dt = DecisionTreeClassifier()
    dt.fit(X, y)
    important_features =  dt.feature_importances_
    features_df = pd.DataFrame({"features" :X.columns, "feature_importance": important_features})
    features = features_df.sort_values(by="feature_importance", ascending=False)
    return features

def module_selection(X_train, y_train, X_test):
    """ selects the best scoring module """
    modules = {
        "gaussian naive_bayes" : GaussianNB, 
        "gradient boosting": GradientBoostingClassifier, 
        "random forest": RandomForestClassifier,
        "logistic regression" : LogisticRegression
    }
    
    scores = {}
    
    for name, module in modules.items():
        module_ = module().fit(X_train, y_train)
        y_pred = module_.predict(X_test)
        print("{} score = {} ".format(name, accuracy_score(y_test, y_pred)))
        scores[name] = module_.score
        
        
    
def engagement_model():
    rec = None
    
    # YOUR CODE HERE
    train = pd.read_csv("assets/train.csv")
    test = pd.read_csv("assets/test.csv")
    train = train.set_index(train["id"])
    test = test.set_index(test["id"])
    
    # prepare data
    X = train.drop(["engagement"], axis=1)
    y = train["engagement"].astype(np.int16)
    # feature selection using Decision_tree
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    # hyper-params
    grid_params = {
        "loss": ["log_loss",  "exponential"],
        "learning_rate" : [10, 1, 0.1, 0.01, 0.001],
        "criterion" : ["friedman_mse", "squared_error"]
                  }
    # best module based on accuracy score: module_selection
    clf = GradientBoostingClassifier()
    
    # grid search for best params, using roc_auc as the metric
    grid_clf_custom = GridSearchCV(clf, param_grid=grid_params, scoring="roc_auc")
    grid_clf_custom.fit(X_train, y_train)
    print('Grid best parameter {}'.format(grid_clf_custom.best_params_))
    print('Grid best score {}'.format(grid_clf_custom.best_score_))
    
    y_score_gb = grid_clf_custom.decision_function(X_test)
    fpr_gb, tpr_gb, _ = roc_curve(y_test, y_score_gb)
    roc_auc_gb = auc(fpr_gb, tpr_gb)
    accuracy_gb = grid_clf_custom.score(X_test, y_test)
    print("accuracy = {:.2f}   AUC = {:.2f}".format(accuracy_gb, roc_auc_gb))
    """
    plt.plot(fpr_gb, tpr_gb, lw=3, alpha=0.7, 
             label='GBC area = {:0.2f})'.format(roc_auc_gb))

    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate (Recall)', fontsize=16)
    plt.plot([0, 1], [0, 1], color='k', lw=0.5, linestyle='--')
    plt.legend(loc="lower right", fontsize=11)
    plt.title('ROC curve: (engagement)', fontsize=16)
    plt.axes().set_aspect('equal')
    """
    
    rec =  pd.Series(grid_clf_custom.predict(test), index=test.index)
    
    return rec
    
    
        
    
    
    
    
    
    #return rec

In [48]:
predictions = engagement_model()
predictions.shape

Grid best parameter {'criterion': 'squared_error', 'learning_rate': 0.1, 'loss': 'log_loss'}
Grid best score 0.890028833006752
accuracy = 0.87   AUC = 0.87


(2309,)

In [49]:
stu_ans = engagement_model()
assert isinstance(stu_ans, pd.Series), "Your function should return a pd.Series. "
assert len(stu_ans) == 2309, "Your series is of incorrect length: expected 2309 "
assert np.issubdtype(stu_ans.index.dtype, np.integer), "Your answer pd.Series should have an index of integer type representing video id."

Grid best parameter {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'log_loss'}
Grid best score 0.8900727171181527
accuracy = 0.87   AUC = 0.87
